# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: create a tower of cubes.

## Instructions
* Solve the problem below.
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a report (maximum 3 pages) in pdf format detailing the methodology you followed to solve the problem. You should add the plots in the report (does not count for the page limit).
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Problem

Implement controller(s) using the code skeletong below to get the robot to build a tower of 3 cubes, with the green cube on the bottom and the blue cube on the top. 

<div>
<img src="./manipulation_scene.jpg" width="400"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to use at least one inverse geometry method
* You need to compensate for the gravity of the robot

In your report, describe the controller(s) and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

## Scene details
The position of the red cube in the spatial frame is $(0.35,0.58,0.65)$, the green cube is at $(0.2,0.58,0.65)$ and the blue cube at $(0.0,0.58,0.65)$.


In [1]:
import numpy as np
import time
import scipy
import matplotlib.pyplot as plt
from utils import Utils

from panda import Simulator, PandaRobot

np.set_printoptions(precision=4, suppress = True)

pybullet build time: Mar 14 2024 13:49:57


# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])
pose = my_robot.FK(q)
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [ 0.      0.      0.     -1.5708  0.      1.5708  0.    ]

 is

 [[-0.7071  0.7071  0.      0.    ]
 [ 0.7071  0.7071 -0.      0.5545]
 [-0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [ 0.482   0.3511  0.6629 -0.1936  0.9883  0.4191  0.6955] 
 
 the gravity force seen on the joints is 
 
 [  0.     -24.2473   1.7638   6.4243   0.5783   1.0095   0.    ]


# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently does not do anything.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [5]:
import scipy.optimize


p_cubes = [
    np.array([0.2,0.58,0.65]),
    np.array([0.35,0.58,0.65]),
    np.array([0.0,0.58,0.65]),
]

Twr = np.block([
    [np.identity(3), np.array([[0.0],[0.0],[0.5]])],
    [0.0,0.0,0.0,1.0]
])

Trw = np.block([
    [np.identity(3), np.array([[0.0],[0.0],[-0.5]])],
    [0.0,0.0,0.0,1.0]
])

# def world_to_robot(pos):
#     return (Trw@np.block([pos,1.0]).T)[0:3]

def world_to_robot(pose):
    T_world_base = np.array([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.5],
        [0.0, 0.0, 0.0, 1.0],
    ])
    
    T_base_world = Utils.inverse_homogeneous_matrix(T_world_base)
    
    return T_base_world @ pose

def error(q,goal):
    current = my_robot.FK(q)[0:3,3]
    error = np.sum( (current-goal)*(current-goal) )
    return error

def position_error(q, p_desired):

    pose = my_robot.FK(q)
    p_current = pose[0:3,3]
    error = p_current-p_desired

    return error.dot(error)


def inverse_kinematics(p_desired):
    
    initial_guess = np.zeros(7)
    result = scipy.optimize.minimize(position_error,initial_guess,args=(p_desired))

    return result.x



In [6]:
joint_limits = [
    [-2.8973, 2.8973],
    [-1.7628, 1.7628],
    [-2.8973, 2.8973],
    [-3.0718, -0.0698],
    [-2.8973, 2.8973],
    [-0.0175, 3.7525],
    [-2.8973, 2.8973],
]

def check_joint_limits(joint_positions,print=False):

    exceeds_limits = False
    
    for i in range(len(joint_positions)):
        position = joint_positions[i]
        limit_min, limit_max = joint_limits[i]

        if position < limit_min or position > limit_max:
            exceeds_limits = True  # Set flag to True if any joint position exceeds its limit

        distance = (position - limit_min)
        distance = min(distance,limit_max - position)
        if print:
            print(f"Joint {i + 1} - Distance to Nearest Limit: {distance}")

    return exceeds_limits  



In [7]:
tasks = [
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.35],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.82],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "grasp",
        "goal": "open",
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.35],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.77],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "grasp",
        "goal": "close"
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.35],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.90],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.90],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.85],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "grasp",
        "goal": "open"
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.90],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.90],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.77],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "grasp",
        "goal": "close"
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.95],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.95],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 0.90],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    },
    {
        "type": "grasp",
        "goal": "open"
    },
    {
        "type": "eef",
        "goal":  np.array([
        [-1.0, 0.0, 0.0, 0.2],
        [0.0, 1.0, 0.0, 0.58],
        [0.0, 0.0, -1.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        ]),
        "max_error": 0.01,
        "P": 50,
        "D": 50,
    }
]

In [8]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation
run_time = 30.

# simulation time step
dt = 0.0001
num_steps = int(run_time/dt)

# we store information
ndofs = 7
time = np.zeros([num_steps])

# the PD gains
task_index = 0
pos_ref = np.array([0.0,0.0,0.0])

holding = False

### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    if task_index >= len(tasks):
        break
    task = tasks[task_index]

    # get the current joint positions/velocities
    q, dq = simulator.get_state()

    match task["type"]:

        case"eef":

            # EEF Jacobian in Body Space
            J = my_robot.get_jacobian(q, 'B')

            # Task Information
            desired_pose_world =  task["goal"]
            desired_pose = world_to_robot(desired_pose_world)
            P = task["P"]*np.identity(6)
            D = task["D"]*np.identity(6)

            # Measured EEF Pose/Twist
            measured_pose = my_robot.FK(q)
            measured_twist = J@dq
            
            # Computed Pose Error (as twist)                 
            delta_poses = Utils.inverse_homogeneous_matrix(measured_pose) @ desired_pose
            pose_error = Utils.transformation_to_twist(delta_poses).squeeze()
            
            # Computed Twist Error        
            reference_twist = np.zeros(6)
            twist_error = reference_twist - measured_twist

            # Positional Error to Stop Changing
            if np.all(abs(pose_error) < task["max_error"]):
                print("Done with task: ", task_index)
                task_index = task_index + 1
                continue
            else:
                print("Error: ", pose_error)

            # Impedence Control
            joint_torques = J.T @ ( P @ pose_error + D@ twist_error) + my_robot.g(q)

            if holding:
                joint_torques += J.T @ np.array([0.0,0.0,0.0,0.0,0.0,-0.981])

            simulator.send_joint_torque(joint_torques)

        
        case "grasp":

            if task["goal"]=="open":
                holding = False
                simulator.gripper_move([0.04,0.04])
            elif task["goal"]=="close":
                holding = True
                simulator.gripper_move([0.0,0.0])

            task_index = task_index + 1
            
    simulator.step()
    #

Version = 4.1 Metal - 88
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2024-04-27 18:10:40.583 Python[80813:887943] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Error:  [ 0.      0.     -0.7854 -0.3218  0.1616  0.3045]
Error:  [ 0.0002 -0.0004 -0.7835 -0.3226  0.1613  0.3044]
Error:  [ 0.0002 -0.0007 -0.7807 -0.3233  0.1608  0.3042]
Error:  [ 0.0002 -0.0009 -0.7775 -0.3239  0.1603  0.3041]
Error:  [ 0.0002 -0.0009 -0.774  -0.3245  0.1597  0.3039]
Error:  [ 0.0002 -0.0009 -0.7704 -0.325   0.159   0.3037]
Error:  [ 0.0001 -0.0009 -0.7667 -0.3254  0.1583  0.3035]
Error:  [-0.     -0.0008 -0.7629 -0.3258  0.1576  0.3033]
Error:  [-0.0001 -0.0006 -0.7592 -0.3261  0.1569  0.303 ]
Error:  [-0.0002 -0.0005 -0.7554 -0.3263  0.1562  0.3027]
Error:  [-0.0003 -0.0003 -0.7516 -0.3265  0.1554  0.3024]
Error:  [-0.0004 -0.0001 -0.7479 -0.3267  0.1547  0.3021]
Error:  [-0.0005  0.0001 -0.7442 -0.3268  0.1539  0.3017]
Error:  [-0.0006  0.0003 -0.7405 -0.3269  0.1531  0.3013]
Error:  [-0.0007  0.0005 -0.7368 -0.3269  0.1523  0.3009]
Error:  [-0.0008  0.0007 -0.7331 -0.3268  0.1516  0.3005]
Error:  [-0.0009  0.0009 -0.7295 -0.3268  0.1507  0.3   ]
Error:  [-0.00

Error:  [-0.0032  0.0047 -0.6378 -0.3129  0.1284  0.2816]
Error:  [-0.0032  0.0048 -0.6346 -0.312   0.1276  0.2808]
Error:  [-0.0033  0.0049 -0.6315 -0.3111  0.1268  0.2799]
Error:  [-0.0034  0.005  -0.6283 -0.3103  0.1261  0.279 ]
Error:  [-0.0034  0.0051 -0.6252 -0.3094  0.1253  0.2781]
Error:  [-0.0035  0.0051 -0.6221 -0.3085  0.1245  0.2772]
Error:  [-0.0035  0.0052 -0.6191 -0.3075  0.1237  0.2763]
Error:  [-0.0036  0.0053 -0.616  -0.3066  0.123   0.2754]
Error:  [-0.0037  0.0054 -0.6129 -0.3056  0.1222  0.2745]
Error:  [-0.0037  0.0054 -0.6099 -0.3047  0.1215  0.2735]
Error:  [-0.0038  0.0055 -0.6069 -0.3037  0.1208  0.2726]
Error:  [-0.0038  0.0056 -0.6039 -0.3027  0.12    0.2717]
Error:  [-0.0039  0.0056 -0.6009 -0.3017  0.1193  0.2707]
Error:  [-0.0039  0.0057 -0.5979 -0.3007  0.1186  0.2698]
Error:  [-0.004   0.0057 -0.595  -0.2996  0.1179  0.2688]
Error:  [-0.004   0.0058 -0.592  -0.2986  0.1172  0.2678]
Error:  [-0.0041  0.0058 -0.5891 -0.2976  0.1165  0.2668]
Error:  [-0.00

In [9]:
# #  here we create a simulation and reset the state of the robot
# simulator = Simulator()
# simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# # we create a robot object so we can use its helper functions
# my_robot = PandaRobot()


# # duration of the simulation
# run_time = 30.

# # simulation time step
# dt = 0.0005
# num_steps = int(run_time/dt)

# # we store information
# ndofs = 7 # number of degrees of freedom (excluding the gripper)
# time = np.zeros([num_steps])

# # the PD gains

# task_index = 0

# ### this is the main control loop
# for i in range(num_steps):
#     # get the current time and save it
#     time[i] = dt * i
    
#     # we get the position and velocities of the joints
#     q, dq = simulator.get_state()

#     check_joint_limits(q,print=False)
#     J = my_robot.get_jacobian(q, 'B')

#     # Measured Position/Velocity
#     measured_pose = my_robot.FK(q)
#     desired_pose_world =  np.array([
#         [-1.0, 0.0, 0.0, 0.33],
#         [0.0, 1.0, 0.0, 0.62],
#         [0.0, 0.0, -1.0, 0.90],
#         [0.0, 0.0, 0.0, 1.0],
#         ])
#     desired_pose = world_to_robot(desired_pose_world)
    
#     print("Measured: \n", measured_pose)
#     print("Desired: \n", desired_pose)
    
#     pose_error = Utils.inverse_homogeneous_matrix(measured_pose) @ desired_pose
#     pose_error = Utils.transformation_to_twist(pose_error).squeeze()
    
#     measured_twist = J@dq
#     reference_twist = np.zeros(6)
#     twist_error = reference_twist - measured_twist

#     # print("Current Twist: \n", measured_twist)
#     # print("Twist Error: \n", twist_error)


#     # Joint Torques Required
#     P = 20*np.identity(6)
#     D = 20*np.identity(6)

#     print("Position Error: ", measured_pose[0:3,3]-desired_pose[0:3,3])

#     joint_torques = J.T @ ( P @ pose_error + D@ twist_error) + my_robot.g(q)



#     # Sending Torque
#     simulator.send_joint_torque(joint_torques)
#     simulator.step()



In [10]:
# # here we create a simulation and reset the state of the robot
# import scipy.linalg


# simulator = Simulator()
# simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])
# simulator.reset_state([0.0,0.0,0.0,0.0,0.0,0.0,0.0])

# # we create a robot object so we can use its helper functions
# my_robot = PandaRobot()

# # duration of the simulation
# run_time = 30.

# # simulation time step
# dt = 0.0001
# num_steps = int(run_time/dt)

# # we store information
# ndofs = 7 # number of degrees of freedom (excluding the gripper)
# time = np.zeros([num_steps])

# task_index = 0
# pos_ref = np.array([0.0,0.0,0.0])


# D = task["D"]*np.array([[1,0,0],[0,1,0],[0,0,1]])
# P = task["P"]*np.array([[1,0,0],[0,1,0],[0,0,1]])


# ### this is the main control loop
# for i in range(num_steps):
#     # get the current time and save it
#     time[i] = dt * i
   

#     # get the current joint positions/velocities
#     q, dq = simulator.get_state()

#     measured_pose = my_robot.FK(q)

#     desired_pose = np.copy(measured_pose)
#     desired_pose[0:3,0:3] = np.array([
#         [-1.0,0.0,0.0],
#         [0.0,1.0,0.0],
#         [0.0,0.0,-1.0],
#     ])

#     delta_pose = Utils.inverse_homogeneous_matrix(measured_pose)@desired_pose
#     twist_skewed = scipy.linalg.logm(delta_pose)
#     twist = Utils.from_skew_6d(twist_skewed)
#     print(twist)

    
#     joint_torques = J.T @ ( P @ twist ) + my_robot.g(q)
#     J = my_robot.get_jacobian(q, 'O')





#     print(my_robot.FK(q))
#     # joint_torques = 20*(q-q_ref)
    
#     # simulator.send_joint_torque(joint_torques)        
#     # simulator.step()
#     #


    

In [11]:
# tasks = [
#     {
#         "type": "eef_position",
#         "goal": np.array([0.33,0.62,0.85]),
#         "max_error": 0.0001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "grasp",
#         "goal": "open"
#     },
#      {
#         "type": "eef_position",
#         "goal": np.array([0.33,0.62,0.72]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "grasp",
#         "goal": "close"
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.33,0.62,0.95]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.2,0.62,0.95]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.2,0.62,0.83]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "grasp",
#         "goal": "open"
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.2,0.62,0.95]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.0,0.62,0.95]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.035,0.60,0.75]),
#         "max_error": 0.001,
#         "P": 50,
#         "D": 50,
#     },
#     {
#         "type": "grasp",
#         "goal": "close"
#     },
#     {
#         "type": "eef_position",
#         "goal": np.array([0.02,0.62,0.95]),
#         "max_error": 0.001,
#         "P": 20,
#         "D": 50,
#     },
    

# ]

In [12]:
# # here we create a simulation and reset the state of the robot
# simulator = Simulator()
# simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# # we create a robot object so we can use its helper functions
# my_robot = PandaRobot()

# # duration of the simulation
# run_time = 30.

# # simulation time step
# dt = 0.0001
# num_steps = int(run_time/dt)

# # we store information
# ndofs = 7
# time = np.zeros([num_steps])

# # the PD gains
# task_index = 0
# pos_ref = np.array([0.0,0.0,0.0])

# ### this is the main control loop
# for i in range(num_steps):
#     # get the current time and save it
#     time[i] = dt * i
#     if task_index >= len(tasks):
#         break
#     task = tasks[task_index]

#     # get the current joint positions/velocities
#     q, dq = simulator.get_state()

#     match task["type"]:

#         case"eef_position":

#             # we get the position and velocities of the joints
#             J = my_robot.get_jacobian(q, 'O')
#             Jw = J[0:3]
#             Jv = J[3:6]

#             # Gains
#             D = task["D"]*np.array([[1,0,0],[0,1,0],[0,0,1]])
#             P = task["P"]*np.array([[1,0,0],[0,1,0],[0,0,1]])

#             # Measured Pose and Twist
#             pose = my_robot.FK(q)
#             print("rotation: ", pose[0:3,0:3])
#             twist = J@dq

#             # Current EEF Position and Velocity
#             pos_measured = pose[0:3,3]
#             vel_measured = twist[3:6]

#             # Desired Position/Velocity
#             pos_ref = world_to_robot(task["goal"]).T
#             vel_ref = np.array([0.0,0.0,0.0]).T

#             # Errors
#             pos_error = pos_ref - pos_measured
#             vel_error = vel_ref - vel_measured
#             print("Position Error: \n ", pos_error)
#             print("Velocity Error: \n", vel_error)

#             if (pos_error.dot(pos_error)<=task["max_error"]):
#                 print("Done with task: ", task_index)
#                 task_index = task_index + 1
#                 continue
#             else:
#                 pos_ref.dot(pos_error)

#             # Joint Torques Required
#             joint_torques = Jv.T @ ( P @ pos_error + D @ vel_error ) + my_robot.g(q)
#             simulator.send_joint_torque(joint_torques)
        
#         case "grasp":

#             if task["goal"]=="open":
#                 simulator.gripper_move([0.04,0.04])
#             elif task["goal"]=="close":
#                 simulator.gripper_move([0.0,0.0])

#             task_index = task_index + 1
            
#     simulator.step()
#     #